<a href="https://colab.research.google.com/github/KRiver28/prac/blob/master/cafe_test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentencepiece

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.layers import Embedding, TimeDistributed
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import sentencepiece as spm
import numpy as np
import pickle

%cd '/content/drive/MyDrive/BIGDATA_STUDY/NLP/chatbot'

/content/drive/MyDrive/BIGDATA_STUDY/NLP/chatbot


In [4]:
# Sub-word 사전 읽어온다.
with open('chatbot_cafe_test.pkl', 'rb') as f:
    word2idx,  idx2word = pickle.load(f)

VOCAB_SIZE = len(idx2word)
EMB_SIZE = 128
LSTM_HIDDEN = 128
MAX_LEN = 15            # 단어 시퀀스 길이
MODEL_PATH = '/content/drive/MyDrive/BIGDATA_STUDY/NLP/chatbot/chatbot_cafetest_trained.h5'

In [5]:
# 데이터 전처리 과정에서 생성한 SentencePiece model을 불러온다.
SPM_MODEL = "/content/drive/MyDrive/BIGDATA_STUDY/NLP/chatbot/chatbot_model.model"
sp = spm.SentencePieceProcessor()
sp.Load(SPM_MODEL)

True

In [6]:
# 워드 임베딩 레이어. Encoder와 decoder에서 공동으로 사용한다.
K.clear_session()
wordEmbedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMB_SIZE)

In [7]:
# Encoder
# -------
encoderX = Input(batch_shape=(None, MAX_LEN))
encEMB = wordEmbedding(encoderX)
encLSTM1 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state = True)
encLSTM2 = LSTM(LSTM_HIDDEN, return_state = True)
ey1, eh1, ec1 = encLSTM1(encEMB)    # LSTM 1층 
_, eh2, ec2 = encLSTM2(ey1)         # LSTM 2층

In [8]:
# Decoder
# -------
# Decoder는 1개 단어씩을 입력으로 받는다.
decoderX = Input(batch_shape=(None, 1))
decEMB = wordEmbedding(decoderX)
decLSTM1 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state=True)
decLSTM2 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state=True)
dy1, _, _ = decLSTM1(decEMB, initial_state = [eh1, ec1])
dy2, _, _ = decLSTM2(dy1, initial_state = [eh2, ec2])
decOutput = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))
outputY = decOutput(dy2)

In [9]:
# Model
# -----
model = Model([encoderX, decoderX], outputY)
model.load_weights(MODEL_PATH)

In [10]:
# Chatting용 model
model_enc = Model(encoderX, [eh1, ec1, eh2, ec2])

ih1 = Input(batch_shape = (None, LSTM_HIDDEN))
ic1 = Input(batch_shape = (None, LSTM_HIDDEN))
ih2 = Input(batch_shape = (None, LSTM_HIDDEN))
ic2 = Input(batch_shape = (None, LSTM_HIDDEN))

dec_output1, dh1, dc1 = decLSTM1(decEMB, initial_state = [ih1, ic1])
dec_output2, dh2, dc2 = decLSTM2(dec_output1, initial_state = [ih2, ic2])

dec_output = decOutput(dec_output2)
model_dec = Model([decoderX, ih1, ic1, ih2, ic2], [dec_output, dh1, dc1, dh2, dc2])

In [11]:
# Question을 입력받아 Answer를 생성한다.
def genAnswer(question):
    question = question[np.newaxis, :]
    init_h1, init_c1, init_h2, init_c2 = model_enc.predict(question)

    # 시작 단어는 <BOS>로 한다.
    word = np.array(sp.bos_id()).reshape(1, 1)

    answer = []
    for i in range(MAX_LEN):
        dY, next_h1, next_c1, next_h2, next_c2 = model_dec.predict([word, init_h1, init_c1, init_h2, init_c2])
        
        # 디코더의 출력은 vocabulary에 대응되는 one-hot이다.
        # argmax로 해당 단어를 채택한다.
        nextWord = np.argmax(dY[0, 0])

        # 예상 단어가 <EOS>이거나 <PAD>이면 더 이상 예상할 게 없다.
        if nextWord == sp.eos_id() or nextWord == sp.pad_id():
            break
        
        # 다음 예상 단어인 디코더의 출력을 answer에 추가한다.
        answer.append(idx2word[nextWord])
        
        # 디코더의 다음 recurrent를 위해 입력 데이터와 hidden 값을
        # 준비한다. 입력은 word이고, hidden은 h와 c이다.
        word = np.array(nextWord).reshape(1,1)
    
        init_h1 = next_h1
        init_c1 = next_c1
        init_h2 = next_h2
        init_c2 = next_c2
        
    return sp.decode_pieces(answer)

def make_question(que_string):
    q_idx = []
    for x in sp.encode_as_pieces(que_string):
        if x in word2idx:
            q_idx.append(word2idx[x])
        else:
            q_idx.append(sp.unk_id())   # out-of-vocabulary (OOV)
    
    # <PAD>를 삽입한다.
    if len(q_idx) < MAX_LEN:
        q_idx.extend([sp.pad_id()] * (MAX_LEN - len(q_idx)))
    else:
        q_idx = q_idx[0:MAX_LEN]
    return q_idx

In [12]:
# Chatting
# dummy : 최초 1회는 모델을 로드하는데 약간의 시간이 걸리므로 이것을 가리기 위함.
def chatting(n=100):
    for i in range(n):
        question = input('Q : ')
        
        if  question == 'quit':
            break
        
        q_idx = make_question(question)
        answer = genAnswer(np.array(q_idx))
        print('A :', answer)

In [13]:
####### Chatting 시작 #######
print("\nSeq2Seq ChatBot (ver. 1.0)")
print("Chatting 모듈을 로드하고 있습니다 ...")


Seq2Seq ChatBot (ver. 1.0)
Chatting 모듈을 로드하고 있습니다 ...


In [14]:
# 처음 1회는 시간이 걸리기 때문에 dummy question을 입력한다.
answer = genAnswer(np.zeros(MAX_LEN))
print("ChatBot이 준비 됐습니다.")

ChatBot이 준비 됐습니다.


In [15]:
# 채팅을 시작한다.
chatting(100)


Q : 팬케이크 아메리카노 세트 구성은 메인메뉴 두 개가 끝인가요?
A : 네 메인메뉴 두가지로 구성됩니다
Q : 테이크아웃으로 주문하면 캔 컵에 담아주시는 건가요?
A : 아니요 따로 담아드립니다
Q : 아이스크림 무제한 토핑 이벤트는 어떻게 참여하는 건가요?
A : 무료 마카롱 1종세트는 구매 하나 변경되었습니다
Q : 혹시 매장에서 사용가능한 와이파이는 비밀번호가 있을까요?
A : 네 000lotte 입력하시면 바로 연결 되실 거예요
Q : 제가 9일 날짜로 예약했는데요, 이름은 #이름# 이고요 주문 수량만 조금 변경할게요.
A : 네 그 팬케이크로 다시 보내드릴게요
Q : 마카롱은 매일 아침에 직접 제작 하고 계신가요?
A : 네 매일 아침에 제조하고 있어서 내일 들어옵니다
Q : 나눠먹는 큐브 와츄원은 몇 가지 맛이 들어가나요?
A : 총 10가지 맛이 있습니다
Q : 아이스크림 케이크에서 특정 맛을 빼거나 교체할 수 있을까요?
A : 죄송하지만 쿠키 마카롱은 계절 딸기맛을 이용해 변경하실 수 있습니다
Q : 플레인스콘에 들어가는 버터 브랜드가 무엇인가요?
A : 네 저희  ⁇ 이 버터를 사용하고 있습니다
Q : quit
